In [1]:
import os
import sys
import time
import threading
from pathlib import Path
from time import sleep
import pyscan as ps
import ipywidgets as widgets
from IPython.display import display

Could not load Keysight SD1
Could not load Keysight SD1
pylablib not found, AttocubeANC350 not loaded
Basler Camera software not found, BaserCamera not loaded
Helios Camera not installed
msl not installed, Thorlabs BSC203 driver not loaded
seabreeze module not found, Ocean Optics not imported
Failed to load spinapi library.
spinapi is not installed, PulseBlaster driver not loaded.
Thorlabs Kinesis not found, ThorlabsBSC203 not loaded
Thorlabs Kinesis not found, ThorlabsBPC303 not loaded
Thorlabs Kinesis not found, ThorlabsMFF101 not loaded


In [2]:
instruments = ps.ItemAttribute()
instruments.ips120 = ps.new_instrument(gpib_address=25)
devices = ps.ItemAttribute()
devices.magnet = ps.OxfordIPS120(instruments.ips120)

In [3]:
devices.magnet

Heater off
At rest
activity: Clamped
get_field() = 0.0
field_set_point = 0.05
field_rate = 0.1

## Issues with original magnet driver in pyscan

In [4]:
# fix clamp()
# need to ensure that the field is at zero before clamping

## Fix magnet operation by adding a new interface (keeping legacy commands for compatibility)

In [8]:
devices.magnet

Heater off
At rest
activity: Clamped
get_field() = 0.0
field_set_point = 0.05
field_rate = 0.1

In [6]:
devices.magnet.local() # keyword: locked=False

In [7]:
devices.magnet.remote() # keyword: locked=False

In [9]:
devices.magnet.heater('off')

In [34]:
devices.magnet.heater('on')
status = devices.magnet.status()
print(status)


system status (operation) : Normal (X1=0)
system status (voltage) : Normal (X2=0)
Activity ; Clamped (A=4)
LOC/REM status ; Remote & Unlocked (C=3)
Heater ; Heater Fault (H=5)
Mode (rate) ; Amps, Immediate, Fast (M1=0)
Mode (sweep) ; At Rest (M2=0)


In [35]:
devices.magnet.instrument.query('X')

'X00A4C3H5M00P04'

In [38]:
status = devices.magnet.status()
if status.H == 5:
    print("switch heater fault")
    _ = devices.magnet.instrument.query("H0")
    sleep(0.5)
    status = devices.magnet.status()
print(status)


system status (operation) : Normal (X1=0)
system status (voltage) : Normal (X2=0)
Activity ; Clamped (A=4)
LOC/REM status ; Remote & Unlocked (C=3)
Heater ; Off Magnet at Zero (H=0)
Mode (rate) ; Amps, Immediate, Fast (M1=0)
Mode (sweep) ; At Rest (M2=0)


In [37]:
devices.magnet.instrument.read()

'X00A4C3H0M00P04'

In [10]:
devices.magnet.get_field()

0.0

In [11]:
devices.magnet.get_persistent_field() # remembers the persistent field when the heater was last turned off

0.0

In [12]:
print(devices.magnet.field_set_point)
print(devices.magnet.field_rate)

0.05
0.1


In [13]:
devices.magnet.to_set_point()

In [14]:
devices.magnet.hold()

In [15]:
devices.magnet.to_zero()

In [14]:
status = devices.magnet.status()
print(status)


system status (operation) : Normal (X1=0)
system status (voltage) : Normal (X2=0)
Activity ; Clamped (A=4)
LOC/REM status ; Remote & Unlocked (C=3)
Heater ; Heater Fault (H=5)
Mode (rate) ; Amps, Immediate, Fast (M1=0)
Mode (sweep) ; At Rest (M2=0)


In [17]:
print(f"remote: {devices.magnet.remote_status()}")
print(f"heater: {devices.magnet.heater_status()}")
print(f"sweeping: {devices.magnet.sweeping_status()}")
print(f"persistent: {devices.magnet.persistent_status()}")

remote: True
heater: True
sweeping: False
persistent: False


## GUI for Oxford IPS120

In [18]:
heater_button = widgets.ToggleButton(description='heater')
field_float = widgets.FloatText(description='field')
activity_buttons = widgets.ToggleButtons(description='activity', options=['hold', 'to set point', 'to zero'])
field_set_point_float = widgets.FloatText(description='set point')
field_rate_float = widgets.FloatText(description='rate')
persistent_button = widgets.ToggleButton(description='persistent', value=False, disabled=True)
persistent_field_float = widgets.FloatText(description='persistent field', value=0.0, disabled=True)
layout = widgets.VBox([
    widgets.HBox([field_set_point_float, field_rate_float]), 
        activity_buttons,
        heater_button, 
        field_float,
        widgets.HBox([persistent_button, persistent_field_float]),
        ])

# relying on a global variable, need to address this
widget_dict = {}
widget_dict['heater_button'] = heater_button
widget_dict['activity_buttons'] = activity_buttons
widget_dict['field_float'] = field_float
widget_dict['field_rate_float'] = field_rate_float
widget_dict['field_set_point_float'] = field_set_point_float
widget_dict['persistent_button'] = persistent_button
widget_dict['persistent_field_float'] = persistent_field_float
start = time.time()
def update(widget_dict, magnet, event):
    '''
    Update the magnet on a schedule
    '''
    slow_update = 2
    fast_update = 0.5
    update_time = 0.1
    t = threading.current_thread()
    slow_time = time.time()-slow_update # do a slow update right away
    fast_time = time.time()-fast_update # do a fast update right away
    while getattr(t, "do_run", True):
        current_time = time.time()
        if (current_time - slow_time) > slow_update:
            slow_time = time.time()
            # slow updates
            event.wait()
            status = magnet.status()
            if status.H == 0:
                widget_dict['heater_button'].value = False
                # enter bad reverse persistent mode
                if (widget_dict['field_float'].value != 0) and not widget_dict['heater_button'].disabled:
                    # must return to 0 before the heater can be turned on
                    widget_dict['heater_button'].disabled = True
                    widget_dict['persistent_button'].value = True
                    widget_dict['persistent_button'].button_style = 'danger'
                    widget_dict['persistent_button'].disabled = False
                    widget_dict['persistent_field_float'].value = 0
                    widget_dict['persistent_field_float'].disabled = False
                # exit bad reverse persisent mode
                if widget_dict['field_float'].value == 0 and  widget_dict['heater_button'].disabled:
                    widget_dict['heater_button'].disabled = False
                    widget_dict['persistent_button'].value = False
                    widget_dict['persistent_button'].button_style = ''
                    widget_dict['persistent_button'].disabled = True
                    widget_dict['persistent_field_float'].value = 0
                    widget_dict['persistent_field_float'].disabled = True
            if status.H == 1:
                widget_dict['heater_button'].value = True
                # exit persistent
                if not widget_dict['persistent_button'].disabled:
                    widget_dict['persistent_button'].value = False
                    widget_dict['persistent_button'].disabled = True
                    widget_dict['persistent_button'].button_style = ''
                    widget_dict['persistent_field_float'].value = 0
                    widget_dict['persistent_field_float'].disabled = True
                    event.wait()
                    magnet.hold()
                    widget_dict['field_set_point_float'].disabled = False
                    # maybe GUI starts in persistent mode
                    try:
                        widget_dict['field_set_point_float'].value = nonpersistent_set_point
                    except NameError:
                        widget_dict['field_set_point_float'].value = magnet.get_field()
                    
            if status.H == 2:
                # only allow the heater to turn on when the field and the persistent field are the same
                event.wait()
                if (widget_dict['field_float'].value != magnet.get_persistent_field()) and not widget_dict['heater_button'].disabled:
                    widget_dict['heater_button'].disabled = True
                event.wait()
                if (widget_dict['field_float'].value == magnet.get_persistent_field()) and widget_dict['heater_button'].disabled:
                    widget_dict['heater_button'].disabled = False
                # enter_persistent
                if not widget_dict['field_set_point_float'].disabled:
                    widget_dict['persistent_button'].value = True
                    widget_dict['persistent_button'].disabled = False
                    widget_dict['persistent_button'].button_style = 'warning'
                    event.wait()
                    widget_dict['persistent_field_float'].value = magnet.get_persistent_field()
                    widget_dict['persistent_field_float'].disabled = False
                    event.wait()
                    nonpersistent_set_point = magnet.field_set_point
                    event.wait()
                    magnet.field_set_point = magnet.get_persistent_field()
                    widget_dict['field_set_point_float'].disabled = True
                
            if status.A == 0:
                # hold
                widget_dict['activity_buttons'].value = "hold"
            elif status.A == 1:
                widget_dict['activity_buttons'].value = "to set point"
            elif status.A == 2:
                widget_dict['activity_buttons'].value = "to zero"
            event.wait()
            widget_dict['field_set_point_float'].value = magnet.field_set_point
            event.wait()
            widget_dict['field_rate_float'].value = magnet.field_rate
        if (current_time-fast_time) > fast_update:
            fast_time = time.time()
            event.wait()
            widget_dict['field_float'].value = magnet.get_field()
        sleep(update_time)
    print("stopping update")
    
# @observe("heater_button")
def heater_button_clicked(change):
    # requires global devices.magnet
    event.clear()
    if change['new']:
        devices.magnet.heater('on')
    else:
        devices.magnet.heater('off')
    event.set()
    
def activity_buttons_clicked(change):
    event.clear()
    match change['new']:
        case "to set point":
            devices.magnet.to_set_point()
        case "to zero":
            devices.magnet.to_zero()
        case "hold":
            devices.magnet.hold()
    event.set()

def field_set_point_float_changed(change):
    event.clear() # pause the thread
    devices.magnet.field_set_point = change['new']
    event.set() # restart the thread

def field_rate_float_changed(change):
    event.clear() # pause the thread
    devices.magnet.field_rate = change['new']
    event.set() # restart the thread
        
#heater_button.on_trait_change(heater_button_clicked)
heater_button.observe(heater_button_clicked, names="value")
activity_buttons.observe(activity_buttons_clicked, names="value")
field_set_point_float.observe(field_set_point_float_changed, names="value")
field_rate_float.observe(field_rate_float_changed, names="value")

In [19]:
event = threading.Event()
event.set()
thread = threading.Thread(target=update, args=(widget_dict, devices.magnet, event))
display(layout)
thread.start()

In [20]:
thread.is_alive()

True

In [21]:
thread.do_run = False

stopping update
